In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import utils.visualizer as uv
import utils.engine as ue

In [ ]:
%load_ext autoreload
%autoreload 2

## 1. 1dice_freezing

In [ ]:
ice_thickness_init = 2.0
Ni = 10
dzi_init = np.full(Ni, ice_thickness_init/Ni)
salinity_init = np.linspace(4.0, 1.0, Ni)
Tis_init = -10.0

time_step=3600.0
Ta=lambda t: -30.0

Ti_func_init = lambda z: (z*Tis_init + (ice_thickness_init-z)*ue.Tf_i(30.0))/ice_thickness_init
ice_cells_init = dzi_init.cumsum() - dzi_init/2

In [ ]:
process_1dice_freezing = ue.main_process\
(
    time_step=time_step,
    time_end=250*time_step,
    N_pseudoiter=25,
    Ti_init=Ti_func_init(ice_cells_init),
    Ts_init=[],
    Tis_init=Tis_init,
    Tsa_init=np.nan,
    dzi_init=dzi_init,
    dzs_init=[],
    salinity=salinity_init,
    snow_thickness_threshold=0.05,
    Toi=lambda t: ue.Tf_i(30.0),
    Ta=Ta,
    p=lambda t: 0.0,
    F_atm_ice = lambda T, time: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(Ta(time) - T),
    F_atm_snow=lambda T, time: 0.0,
    F_sw=lambda t: 0.0,
    F_ocn=lambda T, time: 0.0,
    json_output_folder='test_1dice_freezing/',
    json_ice_prefix='python_1dice_freezing',
    json_snow_prefix=None
)

In [ ]:
uv.animate([process_1dice_freezing])

## 2. 1dice_melting

In [ ]:
ice_thickness_init = 2.0
Ni = 10
dzi_init = np.full(Ni, ice_thickness_init/Ni)
salinity_init = np.linspace(4.0, 1.0, Ni)
Tis_init = -10.0

time_step=3600.0
melt_time_end = 200*time_step
Ta=lambda t: -20.0 + 30.0*t/melt_time_end if t < melt_time_end else 10.0 

Ti_func_init = lambda z: (z*Tis_init + (ice_thickness_init-z)*ue.Tf_i(30.0))/ice_thickness_init
ice_cells_init = dzi_init.cumsum() - dzi_init/2

In [ ]:
process_1dice_melting = ue.main_process\
(
    time_step=time_step,
    time_end=500*time_step,
    N_pseudoiter=25,
    Ti_init=Ti_func_init(ice_cells_init),
    Ts_init=[],
    Tis_init=Tis_init,
    Tsa_init=None,
    dzi_init=dzi_init,
    dzs_init=[],
    salinity=salinity_init,
    snow_thickness_threshold=0.05,
    Toi=lambda t: ue.Tf_i(30.0),
    Ta=Ta,
    p=lambda t: 0.0,
    F_atm_ice = lambda T, time: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(Ta(time) - T),
    F_atm_snow=lambda T, time: 0.0,
    F_sw=lambda t: 0.0,
    F_ocn=lambda T, time: 0.0,
    json_output_folder='test_1dice_melting/',
    json_ice_prefix='python_1dice_melting',
    json_snow_prefix=None
)

## 3. 1dice_cycle

In [ ]:
def linear_year_exp(winter_time, spring_time, summer_time, autumn_time,
                    lowest_temp, highest_temp,
                    lowest_p, highest_p,
                    t):
    t %= (winter_time + spring_time + summer_time + autumn_time)
    if t < winter_time: # winter
        return lowest_temp, highest_p
    elif t < (winter_time + spring_time): # spring
        return lowest_temp + (t - winter_time)*(highest_temp - lowest_temp)/spring_time,\
               highest_p + (t - winter_time)*(lowest_p - highest_p)/spring_time
    elif t < (winter_time + spring_time + summer_time): # summer
        return highest_temp, lowest_p
    else: # autumn
        return highest_temp + (t - (winter_time + spring_time + summer_time))\
                             *(lowest_temp - highest_temp)/autumn_time,\
               lowest_p + (t - (winter_time + spring_time + summer_time))*(highest_p - lowest_p)/autumn_time

In [ ]:
ice_thickness_init = 4.0
Ni = 30
dzi_init = np.full(Ni, ice_thickness_init/Ni)
salinity_init = np.linspace(4.0, 1.0, Ni)
Tis_init = -15.0

time_step=3600.0
winter_time = 350*time_step
spring_time = 200*time_step
summer_time = 150*time_step
autumn_time = 200*time_step
year_time = winter_time + spring_time + summer_time + autumn_time
lowest_temp = -30.0
highest_temp = 10.0
lowest_p = 0.0
highest_p = 0.0
Ta=lambda t: linear_year_exp(winter_time, spring_time, summer_time, autumn_time,
                             lowest_temp, highest_temp,
                             lowest_p, highest_p, t)[0]

Ti_func_init = lambda z: (z*Tis_init + (ice_thickness_init-z)*ue.Tf_i(30.0))/ice_thickness_init
ice_cells_init = dzi_init.cumsum() - dzi_init/2

In [ ]:
process_1dice_cycle = ue.main_process\
(
    time_step=time_step,
    time_end=3*year_time,
    N_pseudoiter=25,
    Ti_init=Ti_func_init(ice_cells_init),
    Ts_init=[],
    Tis_init=Tis_init,
    Tsa_init=None,
    dzi_init=dzi_init,
    dzs_init=[],
    salinity=salinity_init,
    snow_thickness_threshold=0.05,
    Toi=lambda t: ue.Tf_i(30.0),
    Ta=Ta,
    p=lambda t: 0.0,
    F_atm_ice = lambda T, time: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(Ta(time) - T),
    F_atm_snow=lambda T, time: 0.0,
    F_sw=lambda t: 0.0,
    F_ocn=lambda T, time: 0.0,
    json_output_folder='test_1dice_cycle/',
    json_ice_prefix='python_1dice_cycle',
    json_snow_prefix=None
)

## 4. 1dice_1dsnow_freezing

In [ ]:
ice_thickness_init = 2.0
snow_thickness_init = 0.1
Ni = 20
Ns = 2
dzi_init = np.full(Ni, ice_thickness_init/Ni)
dzs_init = np.full(Ns, snow_thickness_init/Ns)
salinity_init = np.linspace(4.0, 1.0, Ni)
Tis_init = -15.0
Tsa_init = -20.0

time_step=3600.0
Ta=lambda t: -30.0

Ti_func_init = lambda z: (z*Tis_init + (ice_thickness_init-z)*ue.Tf_i(30.0))/ice_thickness_init
ice_cells_init = dzi_init.cumsum() - dzi_init/2
Ts_func_init = lambda z: (z*Tsa_init + (snow_thickness_init-z)*Tis_init)/snow_thickness_init
snow_cells_init = dzs_init.cumsum() - dzs_init/2

In [ ]:
process_1dice_1dsnow_freezing = ue.main_process\
(
    time_step=time_step,
    time_end=500*time_step,
    N_pseudoiter=25,
    Ti_init=Ti_func_init(ice_cells_init),
    Ts_init=Ts_func_init(snow_cells_init),
    Tis_init=Tis_init,
    Tsa_init=Tsa_init,
    dzi_init=dzi_init,
    dzs_init=dzs_init,
    salinity=salinity_init,
    snow_thickness_threshold=0.05,
    Toi=lambda t: ue.Tf_i(30.0),
    Ta=Ta,
    p=lambda t: 0.0,
    F_atm_ice=lambda T, time: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(Ta(time) - T),
    F_atm_snow=lambda T, time: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(Ta(time) - T),
    F_sw=lambda t: 0.0,
    F_ocn=lambda T, time: 0.0,
    json_output_folder='test_1dice_1dsnow_freezing/',
    json_ice_prefix='python_ice',
    json_snow_prefix='python_snow'
)

## 5. 1dice_1dsnow_melting

In [ ]:
ice_thickness_init = 2.0
snow_thickness_init = 0.1
Ni = 20
Ns = 2
dzi_init = np.full(Ni, ice_thickness_init/Ni)
dzs_init = np.full(Ns, snow_thickness_init/Ns)
salinity_init = np.linspace(4.0, 1.0, Ni)
Tis_init = -15.0
Tsa_init = -20.0

time_step=3600.0
melt_time_end = 200*time_step
Ta=lambda t: -20.0 + 30.0*t/melt_time_end if t < melt_time_end else 10.0 

Ti_func_init = lambda z: (z*Tis_init + (ice_thickness_init-z)*ue.Tf_i(30.0))/ice_thickness_init
ice_cells_init = dzi_init.cumsum() - dzi_init/2
Ts_func_init = lambda z: (z*Tsa_init + (snow_thickness_init-z)*Tis_init)/snow_thickness_init
snow_cells_init = dzs_init.cumsum() - dzs_init/2

In [ ]:
process_1dice_1dsnow_freezing = ue.main_process\
(
    time_step=time_step,
    time_end=500*time_step,
    N_pseudoiter=25,
    Ti_init=Ti_func_init(ice_cells_init),
    Ts_init=Ts_func_init(snow_cells_init),
    Tis_init=Tis_init,
    Tsa_init=Tsa_init,
    dzi_init=dzi_init,
    dzs_init=dzs_init,
    salinity=salinity_init,
    snow_thickness_threshold=0.05,
    Toi=lambda t: ue.Tf_i(30.0),
    Ta=Ta,
    p=lambda t: 0.0,
    F_atm_ice=lambda T, time: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(Ta(time) - T),
    F_atm_snow=lambda T, time: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(Ta(time) - T),
    F_sw=lambda t: 0.0,
    F_ocn=lambda T, time: 0.0,
    json_output_folder='test_1dice_1dsnow_melting/',
    json_ice_prefix='python_ice',
    json_snow_prefix='python_snow'
)